In [1]:
"""
Transfer All NSIDC Ground Truth with MODIS Matched Data to MySQL
With year/day/day_label extended for indexing
Author: Qi Liu
Date: Oct.09th, 2016
"""

import pandas as pd
from pandas.tools.plotting import scatter_matrix
import numpy as np
import glob
import os
from pandas.io import sql
import MySQLdb
import time

start_time = time.time()

dpath = '/home/mirabot/googledrive/deeplearning/seaiceprj/mpfdata/test/'#'/home/mirabot/googledrive/IceData_DL/initialtest/data/'
rpath = '/home/mirabot/googledrive/deeplearning/seaiceprj/mpfdata/results/'#'/home/mirabot/googledrive/IceData_DL/initialtest/data/'

# dpath = '/home/mirabot/googledrive/IceData_DL/initialtest/aug_raw/'
# rpath = '/home/mirabot/googledrive/IceData_DL/initialtest/aug_raw/'

mode = 2

flist = glob.glob(dpath+'*.csv')
#rlist = glob.glob(rpath+'*.csv')

#mysql connection, install sudo apt-get install python2.7-mysqldb
con = MySQLdb.connect(host="localhost",port=3306,user="mira67",passwd="1234",db="nsidcgt")
if mode == 0:#record NSIDC ground truth data
    print "Write NSIDC GT"
    for tf in flist:
        df = pd.read_csv(tf, sep=",", skiprows=1, names = ["FID","Id","b1","b2","b3","b4","b5","b6","b7","b8","Rowid","MP_F","I_F","W_F",])
        nsplit = os.path.basename(tf).split('_')
        df_len = len(df['FID'])
        site_name = np.chararray((df_len), itemsize=6)
        site_name[:] = nsplit[0]
        year = np.zeros(df_len)
        year[:] = int(nsplit[1][1:5])
        day = np.zeros(df_len)
        day[:] = int(nsplit[1][5:8])
        day_label = np.chararray(df_len, itemsize=1)
        if len(nsplit[1]) == 9:
            day_label[:] = nsplit[1][8]
        else:
            day_label[:] = 'N'
        df['site'] = pd.Series(site_name, index=df.index)
        df['year'] = pd.Series(year, index=df.index)
        df['day'] = pd.Series(day, index=df.index)
        df['day_label'] = pd.Series(day_label, index=df.index)
        print site_name[0]
        #record data to database
        #print df.head()
        #scale
        df = df.replace(-9999, -99990000)
        df['b1'] = df['b1']*0.0001
        df['b2'] = df['b2']*0.0001
        df['b3'] = df['b3']*0.0001
        df['b4'] = df['b4']*0.0001
        df['b5'] = df['b5']*0.0001
        df['b6'] = df['b6']*0.0001
        df['b7'] = df['b7']*0.0001
                
        df.to_sql(con=con, name=site_name[0], if_exists='append', flavor='mysql',index=False)

elif mode == 1:#record MODIS data
    print "Write MOD09"
    for tf in flist:
        print tf
        df = pd.read_csv(tf, sep=",", skiprows=1, names = ["year","month","day","nrow","ncol","qc","cloud","b1","b2","b3","b4","b5","b6","b7"])
        df.to_sql(con=con, name='mod500', if_exists='append', flavor='mysql')

elif mode == 2:#not use pandas
    sql1 = """
    CREATE TABLE {} (
     `year` INT NULL,
     `month` INT NULL,
     `day` INT NULL,
     `nrow` INT NULL,
     `ncol` INT NULL,
     `qc` INT NULL,
     `cloud` INT NULL,
     `b1` DOUBLE NULL,
     `b2` DOUBLE NULL,
     `b3` DOUBLE NULL,
     `b4` DOUBLE NULL,
     `b5` DOUBLE NULL,
     `b6` DOUBLE NULL,
     `b7` DOUBLE NULL);"""
    
    sql2 = """
    LOAD DATA LOCAL INFILE '{}' 
    INTO TABLE {} 
    FIELDS TERMINATED BY ','
    OPTIONALLY ENCLOSED BY '"'
    LINES TERMINATED BY '\\n'
    IGNORE 1 LINES;"""
    
    for tf in flist:
        print tf
        fn = os.path.basename(tf)
        try:
             cursor = con.cursor()
             cursor.execute(sql1.format(fn[:-4]))
             con.commit()
                
             cursor.execute(sql2.format(tf,fn[:-4]))
             con.commit()
        except Exception,e:
             print str(e)
             # Rollback in case there is any error
             con.rollback()
        print 'complete file %s' % (tf)
        
elif mode == 3:#write results to database
    sql = """    
    LOAD DATA LOCAL INFILE '{}' 
    INTO TABLE {}
    FIELDS TERMINATED BY ','
    OPTIONALLY ENCLOSED BY '"'
    LINES TERMINATED BY '\\n'
    IGNORE 1 LINES;"""
    
    for tf in rlist:
        print tf
        try:
             cursor = con.cursor()
             print sql.format(tf)
             cursor.execute(sql.format(tf))
             #con.commit()
        except Exception,e:
             print str(e)
             #Rollback in case there is any error
             con.rollback()
        print 'complete file %s' % (tf)

# disconnect from server
con.close()
os.system('espeak "Congratulations, done with database"')
print("---total time: %s seconds ---" % (time.time() - start_time))

/home/mirabot/googledrive/deeplearning/seaiceprj/mpfdata/test/arr_cafram.csv
complete file /home/mirabot/googledrive/deeplearning/seaiceprj/mpfdata/test/arr_cafram.csv
---total time: 73.8325519562 seconds ---


1